In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
# config = Config(debug=False, sl=30, selected_columns=True, postpond=10)
config = Config(debug=False, sl=200, selected_columns=True, postpond=0)
# config = Config(debug=True, sl=30, selected_columns=True, postpond=10)
torch.cuda.set_device(config.gpu_start); torch.ones((1, 1)).cuda(); # wakeup gpu

In [ ]:
fns_feathers = sorted(config.fn_cycles.glob('cycle*'))[:config.num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
# if not config.selected_columns: config.cont_names = [o for o in cycles.loc[0].columns[2:] if o not in config.dep_var]
idx = fmtr.beginning_index(postpond=config.postpond)

In [ ]:
df_conts = tile_with_noise(cycles, idx, config, normalize=True)
deps = fmtr.get_y(config.dep_var)
deps = pd.concat([deps]*config.mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps, sl=config.sl)

In [ ]:
save_fn = 'tmp/rnn-postponed-checked'
cyc_cont.to_feather(save_fn + '.cycle')
np.savez_compressed(save_fn + '.idx', idx=idx)

In [ ]:
rnndb = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, config.dep_var, valid_idx=config.valid_idx_tile, bs=config.bs, num_workers=0)

In [ ]:
# n_hidden, n_layers = config.n_cont * 5, 3
n_hidden, n_layers = 100, 3
# hidden_p, input_p, embed_p, weight_p = 0,0,0,0
hidden_p, input_p, weight_p = 0.3, 0.6, 0.5

rnn_enc = ContModel1(1, config.n_cont, n_hidden, n_layers, sl=config.sl,
                     hidden_p=hidden_p, input_p=input_p, embed_p=0, weight_p=weight_p)

layers = [3*config.n_cont, 5, 2]
# drops = [0, 0, 0, 0]
drops = [0.1] * 3
model = MultiInputSequentialRNN(rnn_enc, PoolingLinearClassifier(layers, drops)).cuda()
model.reset()

# weight = cyc_cont[dep_var].max().values
# weight = torch.tensor(weight[::-1] / weight.max(), dtype=torch.float32).cuda()

# loss_func = weighted_rnn_mse(weight)
# loss_func = weighted_our_loss(weight)
loss_func = weighted_rnn_mse(torch.tensor([0., 1.]).cuda())
# loss_func = mapd
# loss_func = MSELossFlat()
metrics = [mapd]
learner = Learner(rnndb, model, loss_func=loss_func, metrics=metrics, opt_func=optim.SGD)

from fastai.callbacks import *
# learner.callback_fns += [ShowGraph, partial(SaveModelCallback, name='rnn0')]
learner.callback_fns += [ShowGraph,]
learner.callbacks += [TerminateOnNaNCallback()]
alpha, beta = 2., 1.
# alpha, beta = 1., 1.
# alpha, beta = 0., 0.
learner.callbacks.append(RNNTrainer(learner, config.sl, alpha=alpha, beta=beta))

clip = 0.5
learner.callback_fns.append(partial(GradientClipping, clip=clip))
split_func = lambda x: list(x)
# learner.split(split_func)
learner.split(rnn_classifier_split)

In [ ]:
learner.callback_fns[-1].keywords['clip'] = 0.5
len(learner.layer_groups), learner.layer_groups

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=50, end_lr=100)
learner.recorder.plot(1,1)

In [ ]:
lr = 1e-0
lrs = np.array([0] + [lr/20, lr/10, lr/3] + [lr])
# lrs = np.array(lr)
learner.fit_one_cycle(20, lr)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=50)
learner.recorder.plot(1,1)

In [ ]:
lr = 1e-1
lrs = np.array([0] + [lr/20, lr/10, lr/3] + [lr])
learner.fit_one_cycle(20, lrs)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot()

In [ ]:
lr = 1e-3
lrs = np.array([0] + [lr/20, lr/10, lr/3] + [lr])
learner.fit_one_cycle(100, lrs)

In [ ]:
learner.save('rnn-postpond0')

In [ ]:
learner.load('rnn-postpond0');

In [ ]:
children(learner.model)[0].train();

In [ ]:
learner.model = learner.model.eval()
# learner.model = learner.model.train()

In [ ]:
# it = iter(learner.data.fix_dl)
# it = iter(learner.data.train_dl)
it = iter(learner.data.valid_dl)

In [ ]:
# x, y = ni(learner.data.train_dl)
# x, y = ni(learner.data.valid_dl)
x, y = next(it)
x[1] = x[1].requires_grad_()
learner.model.reset()
# y, p = y.cpu(), learner.model(*x)[0].exp().cpu()
y, p = y.cpu(), learner.model(*x)[0].cpu()
y[:,0], p[:,0] = 1,1
y_np, p_np = to_np(y), to_np(p)
# y_s, p_s = y_np[::mulr], p_np[::mulr]
# our_metrics(y, p), our_metrics_np(y_np, p_np)
# lf = weighted_rnn_mse(weight.cpu())
l = mapd(p, y); l

In [ ]:
s = 0
x_np = to_np(x[1][s]).reshape(config.sl, config.n_cont)
x_np.shape
for feat in range_of(x_np[0]):
#     plt.figure()
    ax = plt.plot(x_np[:,feat])
#     legend = plt.legend(config.cont_names[feat])
#     set_font(legend.texts)

In [ ]:
ax = cycles.loc[s,config.cont_names].iloc[idx[s]:idx[s]+config.sl].plot()
set_font(ax.legend().texts)

In [ ]:
i = 0
plt.scatter(y_np[:,i], p_np[:,i]);

In [ ]:
i = 1
plt.scatter(y_np[:,i], p_np[:,i]);

In [ ]:
# last timestep
i, c, t = 0, 1, 29
d = torch.autograd.grad(l, x[1], retain_graph=True)
d = d[0][i,t*n_cont:(t+1)*n_cont]
d_np = to_np(d)

In [ ]:
# average along time axis
i, c = 3, 1
d = torch.autograd.grad(p[i,c], x[1], retain_graph=True)
d = d[0][i].view(n_cont, -1)
d_np = to_np(d)
d_np = np.abs(d_np)
d_np = d_np.mean(1)

In [ ]:
impact = d_np.argsort()[::-1]
cs = columns_x[impact]
pprint(list(enumerate(zip(impact, d_np[impact], cs))))
plots(cycles[i].loc[:,cs[:10]].iloc[idx[i]-50:idx[i]+100]);

In [ ]:
# l x b x t x h
l, b = 0, 0
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for t in range(30)]

In [ ]:
# l x b x t x h
l, b, t = 0, 0, 1
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for l in range(n_layers)]

In [ ]:
list(zip(y_np.tolist(), p_np.tolist()))

In [ ]:
i = 0
plt.scatter(_s[:,i], y_s[:,i])

In [ ]:
i = 1
plt.scatter(p_s[:,i], y_s[:,i])